In [1]:
import json
from typing import Union
import numpy as np
import pandas as pd
import plotly.express as px
from process_base import X_t_proportional, X_t_stoploss

In [2]:
def get_ruin_of_instance(X_t: Union[X_t_proportional, X_t_stoploss], process_config: dict, max_time: float, seed: int):
    # update seed
    process_config.update(seed=seed)
    # make process instance
    x_t = X_t(**process_config)
    # generate process until max_time
    x_t.get_value_at_time(time=max_time)
    # get ruin probability
    for i_process in range(process_config['n'] + 1):
        for arr_time in x_t.inter_arrival_times[i_process]:
            if arr_time <= max_time:
                value = x_t.get_value_at_time(time=arr_time)
                if value[0] < 0 or value[i_process] < 0:
                    return True
    return False


def Psi(u: list, X_t: Union[X_t_proportional, X_t_stoploss], process_config: dict, max_time: float, n_simulations: int=1000, seed_offset: int=0):
    # set initial capital
    process_config.update(u=u)
    is_ruin = [False] * n_simulations
    for i_sim in range(n_simulations):
        is_ruin[i_sim] = get_ruin_of_instance(X_t, process_config, max_time, i_sim + seed_offset)
    return sum(is_ruin) / n_simulations

# Case 1: Proportional reinsurance

In [21]:
config_proportional = json.load(open("./configs/proportional/config_1.json"))
config_proportional

{'model_settings': {'n': 1,
  'c': [0.924, 0.756],
  'alpha': [0.6],
  'poisson_rate': [0.6, 1.2],
  'claims_rate': [1.5, 1.0]},
 's': [1.0, 0.5]}

In [22]:
x_t = X_t_proportional(
    u=[10, 5],
    n=config_proportional["model_settings"]["n"],
    c=config_proportional["model_settings"]["c"],
    alpha=config_proportional["model_settings"]["alpha"],
    poisson_rate=config_proportional["model_settings"]["poisson_rate"],
    claims_rate=config_proportional["model_settings"]["claims_rate"],
    seed=42
)

In [24]:
x_t.plot_process(max_time=1000)

In [25]:
%%time

tmp_config = {
    'n': config_proportional["model_settings"]["n"],
    'c': config_proportional["model_settings"]["c"],
    'alpha': config_proportional["model_settings"]["alpha"],
    'poisson_rate': config_proportional["model_settings"]["poisson_rate"],
    'claims_rate': config_proportional["model_settings"]["claims_rate"]
}

Psi(u=[10, 5], X_t=X_t_proportional, process_config=tmp_config, max_time=1000, n_simulations=1000, seed_offset=0)

CPU times: user 10.6 s, sys: 89.4 ms, total: 10.7 s
Wall time: 10.8 s


0.696

# Case 2: Stop-loss reinsurance

In [26]:
config_stoploss = json.load(open("./configs/stoploss/config_1.json"))
config_stoploss

{'model_settings': {'n': 1,
  'c': [1.68, 1.26],
  'b': [50.0],
  'poisson_rate': [0.6, 1.2],
  'claims_rate': [1.5, 1.0]},
 's': [1.0, 0.5]}

In [27]:
x_t = X_t_stoploss(
    u=[10, 5],
    n=config_stoploss["model_settings"]["n"],
    c=config_stoploss["model_settings"]["c"],
    b=config_stoploss["model_settings"]["b"],
    poisson_rate=config_stoploss["model_settings"]["poisson_rate"],
    claims_rate=config_stoploss["model_settings"]["claims_rate"],
    seed=42
)

In [31]:
x_t.plot_process(max_time=100)

In [30]:
%%time

tmp_config = {
    'n': config_stoploss["model_settings"]["n"],
    'c': config_stoploss["model_settings"]["c"],
    'b': config_stoploss["model_settings"]["b"],
    'poisson_rate': config_stoploss["model_settings"]["poisson_rate"],
    'claims_rate': config_stoploss["model_settings"]["claims_rate"]
}

Psi(u=[10, 5], X_t=X_t_stoploss, process_config=tmp_config, max_time=1000, n_simulations=1000, seed_offset=0)

CPU times: user 13.5 s, sys: 85.7 ms, total: 13.6 s
Wall time: 13.8 s


0.497